In [12]:
!nvidia-smi

Mon Apr 24 04:57:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    33W /  70W |   1409MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-s_7aa1kc/farm-haystack_bdbfc2ad36cb469daef1116a54c415b8
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-s_7aa1kc/farm-haystack_bdbfc2ad36cb469daef1116a54c415b8
  Resolved https://github.com/deepset-ai/haystack.git to commit 7db025a97bd9b5cce611988a245ffea4790281a0
  Installing build dependencies ... done
  Getting requirements to

In [14]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "data/"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"

In [15]:
reader.save(directory = "my_model")

In [17]:
reader.train(data_dir=data_dir, train_filename="train-v2.0.json", use_gpu=True, n_epochs=1, save_dir="my_model")

Preprocessing dataset:   0%|          | 0/38 [00:00<?, ? Dicts/s]

  0%|          | 0/14427 [00:00<?, ?it/s]

In [18]:
new_reader = FARMReader(model_name_or_path="my_model")

In [26]:
reader_eval_results = new_reader.eval_on_file("data/", "answers.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.


Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [32]:
reader_eval_results

{'EM': 0.0,
 'f1': 0.0,
 'top_n_accuracy': 0.0,
 'top_n': 4,
 'EM_text_answer': 0.0,
 'f1_text_answer': 0.0,
 'top_n_accuracy_text_answer': 0.0,
 'top_n_EM_text_answer': 0.0,
 'top_n_f1_text_answer': 0.0,
 'Total_text_answer': 2,
 'EM_no_answer': 0,
 'f1_no_answer': nan,
 'top_n_accuracy_no_answer': nan,
 'Total_no_answer': 0}

In [99]:
with open ("/content/data/OVEN.txt", "r") as file:
  context = file.read()

In [62]:
# context = ''' ow many bosch oven do we have ? we have 6 oven.
# name of ovens by bosch HBF010BR0Z Series 2, HBF011BR0Z Series 2 , HBF113BR0Z Series 2 , HBA534BS0Z Series 4 , HBA574BR0Z Series 4 , HSG7361B1 Series 8 

# price of HBF010BR0Z Series 2 is 69,990.
# price of HBF011BR0Z Series 2 is 74,790.
# price of HBF113BR0Z Series 2 is 89,990.
# price of HBA534BS0Z Series 4 is 105,990.
# price of HBA574BR0Z Series 4 is 119,990.
# price of HSG7361B1 Series 8 is 269,990.'''

In [101]:
from haystack import Pipeline, Document
from haystack.utils import print_answers
reader = FARMReader(model_name_or_path="/content/my_model")
p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])
res = p.run(
    query="how to check your power to bosch oven?", documents=[Document(content=context)]
)
print_answers(res,details="medium")
#print(res)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: how to check your power to bosch oven?'
'Answers:'
[   {   'answer': 'your control panel',
        'context': 'ost ovens can be recalibrated in increments of 35°F at a '
                   'time via your control panel Check your owner’s manual for '
                   'the exact calibration steps for you',
        'score': 0.5201160907745361},
    {   'answer': 'bake test',
        'context': "uld be causing this?Check your oven's baking capability "
                   'with a simple "bake test" This is one of the many quality '
                   'tests that each Bosch oven undergoes',
        'score': 0.5044004321098328},
    {   'answer': 'Check the main supply connection',
        'context': 'and to resolve it, here are the steps that you can follow: '
                   'Check the main supply connection and ensure that the '
                   'supply is within normal operating rang',
        'score': 0.41355323791503906}]
